##### for cleaning the CPU ram
import gc
gc.collect()

%reset -f

In [32]:
#for cleaning the GPU ram
import torch 
torch.cuda.empty_cache()

In [33]:
TOTAL_ITER = 1000
VALID_EVE =100

In [34]:
import h5py
import json
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import argparse#***

In [35]:
def invertDict(_dict):
    return {v: k for k, v in _dict.items()}

In [36]:
class ClevrDialogDataset(Dataset):
    def __init__(self, dataPath, vocabPath, split, indStart=0, indEnd=-1):
        super(ClevrDialogDataset, self).__init__()
        self.data = h5py.File(dataPath, "r")
        with open(vocabPath, "r") as f:
            self.vocab = json.load(f)
        self.vocab["idx_text_to_token"] = invertDict(self.vocab["text_token_to_idx"])
        self.vocab["idx_prog_to_token"] = invertDict(self.vocab["prog_token_to_idx"])
        self.vocab["idx_prog_to_token"] = invertDict(self.vocab["prog_token_to_idx"])
        self.lenVocabText = len(self.vocab["text_token_to_idx"])
        self.lenVocabProg = len(self.vocab["prog_token_to_idx"])

        self.split = split
        self.indStart = indStart
        self.indEnd = indEnd
        self.maxSamples = indEnd - indStart
        self.maxLenProg = 6

    def __len__(self):
        raise NotImplementedError

    def __getitem__(self, index):
        raise NotImplementedError

In [37]:
class ClevrDialogUnifiedDataset(ClevrDialogDataset):
    def __init__(self, dataPath, vocabPath, split, name, train=True, indStart=0, indEnd=-1):
        super(ClevrDialogUnifiedDataset, self).__init__(dataPath, vocabPath, split, indStart=indStart, indEnd=indEnd)
        #self.captions = torch.LongTensor(np.asarray(self.data["captions"], dtype=np.int64)[indStart: indEnd])
        #self.captionsPrgs = torch.LongTensor(np.asarray(self.data["captionProgs"], dtype=np.int64)[indStart: indEnd])

        self.questions = torch.LongTensor(np.asarray(self.data["questions"], dtype=np.int64)[indStart: indEnd])
        self.quesProgs = torch.LongTensor(np.asarray(self.data["questionProgs"], dtype=np.int64)[indStart: indEnd])
        self.questionRounds = torch.LongTensor(np.asarray(self.data["questionRounds"], dtype=np.int64)[indStart: indEnd])
        self.questionImgIdx = torch.LongTensor(np.asarray(self.data["questionImgIdx"], dtype=np.int64)[indStart: indEnd])
        self.histories = torch.LongTensor(np.asarray(self.data["histories"], dtype=np.int64)[indStart: indEnd])
        self.historiesProgs = torch.LongTensor(np.asarray(self.data["historiesProg"], dtype=np.int64)[indStart: indEnd])

        self.answers = torch.LongTensor(np.asarray(self.data["answers"], dtype=np.int64)[indStart: indEnd])
        self.name = name
        self.train = train

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        assert idx < len(self)
        question = self.questions[idx]
        questionPrg = self.quesProgs[idx]
        questionImgIdx = self.questionImgIdx[idx]
        questionRound = self.questionRounds[idx]

        history = self.histories[idx]
        historiesProg = self.historiesProgs[idx]

        answer = self.answers[idx]
        if self.train:
            #return caption, captionPrg, question, questionPrg, questionRound, history, answer
            return question, questionPrg, questionRound, history,historiesProg, answer
        else:
            return question, questionPrg, questionRound, history,historiesProg, answer, questionImgIdx

In [38]:
COLORS = ["blue", "brown", "cyan", "gray", "green", "purple", "red", "yellow"]
MATERIALS = ["rubber", "metal"]
SHAPES = ["cube", "cylinder", "sphere"]
SIZES = ["large", "small"]

ATTRIBUTES_ALL = COLORS + MATERIALS + SHAPES + SIZES

ANSWER_CANDIDATES = {
    # Count questions
    "count-all": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],
    "count-other": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "count-all-group": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],
    "count-attribute": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],
    "count-attribure-group": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],
    "count-obj-rel-imm": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "count-obj-rel-imm2": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "count-obj-rel-early": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "count-obj-exclude-imm": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "count-obj-exclude-early": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],

    # Existence questions
    "exist-other": ["yes", "no"],
    "exist-attribute": ["yes", "no"],
    "exist-attribute-group": ["yes", "no"],
    "exist-obj-rel-imm": ["yes", "no"],
    "exist-obj-rel-imm2": ["yes", "no"],
    "exist-obj-rel-early": ["yes", "no"],
    "exist-obj-exclude-imm": ["yes", "no"],
    "exist-obj-exclude-early": ["yes", "no"],

    # Seek questions
    "seek-attr-imm": ATTRIBUTES_ALL,
    "seek-attr-imm2": ATTRIBUTES_ALL,
    "seek-attr-early": ATTRIBUTES_ALL,
    "seek-attr-sim-early": ATTRIBUTES_ALL,
    "seek-attr-rel-imm": ATTRIBUTES_ALL,
    "seek-attr-rel-early": ATTRIBUTES_ALL,
}



In [39]:
import json
import numpy as np


def merge_captions_question_programs(path_cap, path_ques, caption_first=True):
    with open(path_cap, "r"):
        c_progs = path_cap.readlines()
    with open(path_ques, "r"):
        q_progs = path_ques.readlines()

    all_merged_progs = []
    i = 0
    while i < len(q_progs):
        cap_idx = i % 11 if caption_first else i % 10
        start_idx_p = i + 1 if caption_first else i
        end_idx_p = start_idx_p + 12 if caption_first else  start_idx_p + 11
        temp = c_progs[cap_idx] + q_progs[start_idx_p, end_idx_p]
        all_merged_progs.append(temp)
        i = end_idx_p


def load_clevr_scenes(scenes_json):
    with open(scenes_json) as f:
        scenes_raw = json.load(f)
    if type(scenes_raw) == dict:
        scenes_raw = scenes_raw["scenes"]

    scenes = []
    for s in scenes_raw:
        table = []
        for i, o in enumerate(s['objects']):
            item = {}
            item['id'] = '%d-%d' % (s['image_index'], i)
            if '3d_coords' in o:
                item['position'] = [np.dot(o['3d_coords'], s['directions']['right']),
                                    np.dot(o['3d_coords'], s['directions']['front']),
                                    o['3d_coords'][2]]
            else:
                item['position'] = o['position']
            item['color'] = o['color']
            item['material'] = o['material']
            item['shape'] = o['shape']
            item['size'] = o['size']
            table.append(item)
        scenes.append(table)
    return scenes


def load_minecraft_scenes(scenes_json):
    with open(scenes_json) as f:
        scenes_raw = json.load(f)
    if type(scenes_raw) == dict:
        scenes_raw = scenes_raw["scenes"]

    scenes = []
    for s in scenes_raw:
        table = []
        for i, o in enumerate(s['objects']):
            item = {}
            item['id'] = '%d-%d' % (s['image_index'], i)
            if '3d_coords' in o:
                item['position'] = [np.dot(o['3d_coords'], s['directions']['right']),
                                    np.dot(o['3d_coords'], s['directions']['front']),
                                    o['3d_coords'][2]]
            else:
                item['position'] = o['position']
            item['nature'] = o['nature']
            item['class'] = o['class']
            item['direction'] = "facing_"
            if o['direction'] == "front":
                item['direction'] += "forward"
            elif o['direction'] == "back":
                item['direction'] += "backward"
            elif o['direction'] == "right":
                item['direction'] += "right"
            elif o['direction'] == "left":
                item['direction'] += "left"
            table.append(item)
        scenes.append(table)
    return scenes

In [40]:
import numpy as np
from copy import deepcopy


#from executor.clevr_statics import COLORS, MATERIALS, SHAPES, SIZES
#from executor.clevr_statics import ANSWER_CANDIDATES as ANSWER_CANDIDATES_CLEVR
#from executor.clevr_statics import ATTRIBUTES_ALL as ATTRIBUTES_ALL_CLEVR

#from utils_m import load_clevr_scenes


class SymbolicExecutorClevr(object):
    """Symbolic executor for clevr-dialog
    """
    def __init__(self, scenesPath):
        super(SymbolicExecutorClevr, self).__init__()
        self.functions = {}
        self.registerFunctions()
        self.uniqueObjFlag = False
        self.colors = COLORS
        self.materials = MATERIALS
        self.shapes = SHAPES
        self.sizes = SIZES
        self.answer_candidates = ANSWER_CANDIDATES#***
        self.attribute_all = ATTRIBUTES_ALL#***
        self.scenes = load_clevr_scenes(scenesPath)

    def reset(self, sceneIdx):
        """Resets the scene

        Args:
            sceneIdx: The index of the new scene
        """
        self.scene = self.scenes[sceneIdx]
        for _obj in self.scene:
            _obj["identifier"] = None
        # store previous objects in a list to better answer
        # xxx-imm, xxx-imm2, xxx-group and xxx-early questions.
        self.objs = []
        self.groups = []
        self.visited = []
        self.currentObj = None
        self.currentGrp = []
        self.uniqueObjFlag = False

    def registerFunctions(self):
        """Registers the available functions of the executor.
        """
        # Captions - extreme location
        self.functions["extreme-right"] = self.extremeRight
        self.functions["extreme-left"] = self.extremeLeft
        self.functions["extreme-behind"] = self.extremeBehind
        self.functions["extreme-front"] = self.extremeFront
        self.functions["extreme-center"] = self.extremeCenter

        # Captions - multiple objects
        self.functions["count-att"] = self.countAttributeCaption

        # Captions - object relations
        self.functions["obj-relation"] = self.objRelation

        # Captions - unique object
        self.functions["unique-obj"] = self.uniqueObject

        # Questions - Count
        self.functions["count-all"] = self.countAll
        self.functions["count-other"] = self.countOther
        self.functions["count-all-group"] = self.countAllGroup
        self.functions["count-attribute"] = self.countAttribute
        self.functions["count-attribute-group"] = self.countAttributeGroup
        self.functions["count-obj-rel-imm"] = self.countObjRelImm
        self.functions["count-obj-rel-imm2"] = self.countObjRelImm2
        self.functions["count-obj-rel-early"] = self.countObjRelEarly
        self.functions["count-obj-exclude-imm"] = self.countObjExcludeImm
        self.functions["count-obj-exclude-early"] = self.countObjExcludeEarly

        # Questions - Exist
        self.functions["exist-other"] = self.existOther
        self.functions["exist-attribute"] = self.existAttribute
        self.functions["exist-attribute-group"] = self.existAttributeGroup
        self.functions["exist-obj-rel-imm"] = self.existObjRelImm
        self.functions["exist-obj-rel-imm2"] = self.existObjRelImm
        self.functions["exist-obj-rel-early"] = self.existObjRelEarly
        self.functions["exist-obj-exclude-imm"] = self.existObjExcludeImm
        self.functions["exist-obj-exclude-early"] = self.existObjExcludeEarly

        # Questions - Seek
        self.functions["seek-attr-imm"] = self.seekAttrImm
        self.functions["seek-attr-imm2"] = self.seekAttrImm
        self.functions["seek-attr-early"] = self.seekAttributeEarly
        self.functions["seek-attr-rel-imm"] = self.seekAttributeRelImm
        self.functions["seek-attr-rel-early"] = self.seekAttributeRelEarly


    ########################################################
    #                   Helper functions                   #
    ########################################################
    def getAttributeType(self, attribute):
        assert attribute in self.attribute_all, "The attribute {} is unkown".format(
            attribute)
        if attribute in self.colors:
            return "color"
        elif attribute in self.materials:
            return "material"
        elif attribute in self.shapes:
            return "shape"
        elif attribute in self.sizes:
            return "size"

    def execute(self, functionLabel, functionArgs):
        assert functionLabel in self.functions, "{} is not a valid function".format(
            functionLabel)
        function = self.functions[functionLabel]
        answer = function(*functionArgs)
        return answer

    def updateCurrentObj(self, obj):
        self.currentObj = obj
        objsCopy = deepcopy(self.objs)
        for i, _obj in enumerate(objsCopy):
            if _obj["id"] == obj["id"]:
                del self.objs[i]
        # Current obj is always kept at the end of the visited objs
        self.objs.append(obj)

    def updateVisited(self, obj):
        if len(self.visited) == 0:
            self.visited.append(obj)
        else:
            newObjFlag = True
            for _obj in self.visited:
                if _obj["id"] == obj["id"]:
                    newObjFlag = False
                    break
            if newObjFlag:
                self.visited.append(obj)

    def getOther(self):
        others = []
        if len(self.visited) < len(self.scene):
            for _obj in self.scene:
                notExisting = True
                for __obj in self.visited:
                    if __obj["id"] == _obj["id"]:
                        notExisting = False
                        break
                if notExisting:
                    others.append(_obj)
        return others

    def updateIdentifier(self, obj, attribute):
        if obj["identifier"] is None:
            obj["identifier"] = attribute
        else:
            identifiers = obj["identifier"].split("-")
            if attribute not in identifiers:
                identifiers.append(attribute)
                obj["identifier"] = "-".join(identifiers)


    ########################################################
    #                   Caption programs                   #
    ########################################################

    def extremeRight(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))

        leftToRight = deepcopy(self.scene)
        leftToRight.sort(key=lambda o: o["position"][0])
        extremeRightObj = leftToRight[-1]
        for attributeType, attribute in zip(attributeTypes, attributes):
            assert extremeRightObj[attributeType] == attribute
            self.updateIdentifier(extremeRightObj, attribute)

        self.updateCurrentObj(extremeRightObj)
        self.updateVisited(extremeRightObj)
        del leftToRight

    def extremeLeft(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))

        leftToRight = deepcopy(self.scene)
        leftToRight.sort(key=lambda o: o["position"][0])
        extremeLeftObj = leftToRight[0]
        for attributeType, attribute in zip(attributeTypes, attributes):
            assert extremeLeftObj[attributeType] == attribute
            self.updateIdentifier(extremeLeftObj, attribute)

        self.updateCurrentObj(extremeLeftObj)
        self.updateVisited(extremeLeftObj)
        del leftToRight

    def extremeFront(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))

        backToFront = deepcopy(self.scene)
        backToFront.sort(key=lambda o: o["position"][1])
        extremeFrontObj = backToFront[-1]
        for attributeType, attribute in zip(attributeTypes, attributes):
            assert extremeFrontObj[attributeType] == attribute
            self.updateIdentifier(extremeFrontObj, attribute)

        self.updateCurrentObj(extremeFrontObj)
        self.updateVisited(extremeFrontObj)
        del backToFront

    def extremeBehind(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))

        backToFront = deepcopy(self.scene)
        backToFront.sort(key=lambda o: o["position"][1])
        extremeBehindObj = backToFront[0]
        for attributeType, attribute in zip(attributeTypes, attributes):
            assert extremeBehindObj[attributeType] == attribute
            self.updateIdentifier(extremeBehindObj, attribute)

        self.updateCurrentObj(extremeBehindObj)
        self.updateVisited(extremeBehindObj)
        del backToFront

    def extremeCenter(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))
        numObjs = len(self.scene)

        frontToBack = deepcopy(self.scene)
        frontToBack.sort(key=lambda o: o["position"][1], reverse=True)

        rightToLeft = deepcopy(self.scene)
        rightToLeft.sort(key=lambda o: o["position"][0], reverse=True)

        prelimenaryCandidates = []

        for i, objFrontToBack in enumerate(frontToBack):
            numObjsInFront = i
            numObjsBehind = len(rightToLeft) - i - 1
            if numObjsInFront <= numObjs / 2 and numObjsBehind <= numObjs / 2:
                prelimenaryCandidates.append(objFrontToBack)
        foundCenter = False
        for _obj in prelimenaryCandidates:
            for i, objRightToLeft in enumerate(rightToLeft):
                if _obj["id"] == objRightToLeft["id"]:
                    numObjsToTheRight = i
                    numObjsToTheLeft = len(frontToBack) - i - 1
                    if numObjsToTheRight <= numObjs / 2 and numObjsToTheLeft <= numObjs / 2:
                        foundCenter = True
                        for attributeType, attribute in zip(attributeTypes, attributes):
                            if _obj[attributeType] != attribute:
                                foundCenter = False
                                break
                        break
            if foundCenter:
                break
        for attributeType, attribute in zip(attributeTypes, attributes):
            self.updateIdentifier(_obj, attribute)
        self.updateCurrentObj(_obj)
        self.updateVisited(_obj)
        del rightToLeft, frontToBack

    def countAttributeCaption(self, attribute):
        attributeType = self.getAttributeType(attribute)
        objs = []
        for _obj in self.scene:
            if _obj[attributeType] == attribute:
                objs.append(deepcopy(_obj))
        for _obj in objs:
            self.updateIdentifier(_obj, attribute)
        # update the current group
        self.currentGrp = objs

        # update the visited objects list
        for _obj in objs:
            self.updateVisited(_obj)

    def getAnchorAttribute(self, attribute_1, attribute_2, scene):
        # The anchor object is unique. If we filter the object list
        # based on the attribute anchor, we must find only one object.
        filterAttribute_1 = self.filterAttribute(scene, attribute_1)
        if len(filterAttribute_1) == 1:
            return attribute_1
        else:
            return attribute_2

    def objRelation(self, attribute, attributeAnchor, relation):
        assert relation in ["left", "right", "front", "behind"]
        # find the anchor object
        if attributeAnchor != self.getAnchorAttribute(attribute, attributeAnchor, self.scene):
            temp = deepcopy(attribute)
            attribute = deepcopy(attributeAnchor)
            attributeAnchor = temp
            if relation == "left":
                relation = "right"
            elif relation == "right":
                relation = "left"
            elif relation == "behind":
                relation = "front"
            elif relation == "front":
                relation = "behind"

        # Order the objects in the scene w.r.t. the relation
        sceneCopy = deepcopy(self.scene)

        if relation in ["left", "right"]:
            sceneCopy.sort(key=lambda o: o["position"][0])
        else:
            sceneCopy.sort(key=lambda o: o["position"][1])

        # get the anchor object
        attributeTypeAnchor = self.getAttributeType(attributeAnchor)
        for i, _obj in enumerate(sceneCopy):
            if _obj[attributeTypeAnchor] == attributeAnchor:
                break
        # save the anchor object before the main object
        anchorObj = _obj
        self.updateIdentifier(anchorObj, attributeAnchor)
        self.updateCurrentObj(anchorObj)
        self.updateVisited(anchorObj)

        if relation in ["left", "behind"]:
            sceneCopy = list(reversed(sceneCopy[:i]))
        else:
            sceneCopy = sceneCopy[i+1:]

        attributeType = self.getAttributeType(attribute)
        # get the main object
        for _obj in sceneCopy:
            # and not equalDicts(_obj, anchorObj):
            if _obj[attributeType] == attribute:
                break
        self.updateIdentifier(_obj, attribute)
        self.updateCurrentObj(_obj)
        self.updateVisited(_obj)
        del sceneCopy

    def uniqueObject(self, *attributes):
        attributes = list(attributes)
        attributeTypes = list(
            map(lambda att: self.getAttributeType(att), attributes))

        for _obj in self.scene:
            found = True
            for attributeType, attribute in zip(attributeTypes, attributes):
                if _obj[attributeType] != attribute:
                    found = False
                    break

            if found:
                break
        for att in attributes:
            self.updateIdentifier(_obj, att)

        self.updateCurrentObj(_obj)
        self.updateVisited(_obj)

    ######################################## Question Programs ########################################
    def filterOutObj(self, scene, obj):
        sceneCopy = deepcopy(scene)
        for i, _obj in enumerate(scene):
            if obj["id"] == _obj["id"]:
                break
        del sceneCopy[i]
        return sceneCopy

    def filterAttribute(self, scene, attribute):
        attributeType = self.getAttributeType(attribute)
        filtered = []
        if len(scene) == 0:
            return filtered

        for _obj in scene:
            if _obj[attributeType] == attribute:
                filtered.append(_obj)
        return filtered

    def excludeAttribute(self, scene, obj, attributeType):
        filtered = []
        if len(scene) == 0:
            return filtered
        for _obj in scene:
            if _obj["id"] != obj["id"] and obj[attributeType] == _obj[attributeType]:
                filtered.append(_obj)

        # Update the visited objects list
        if len(filtered) > 0:
            for _obj in filtered:
                self.updateVisited(_obj)
        return filtered

    def filterLeft(self, scene, obj):
        filtered = []
        if len(scene) == 0:
            return filtered

        for _obj in self.scene:
            # if the x-coordinate of _obj is smaller than the x-coordinate of slef.currentObj,
            # then _obj is located to the left of self.currentObj
            if _obj["position"][0] < obj["position"][0] and _obj["id"] != obj["id"]:
                filtered.append(_obj)
        return filtered

    def filterRight(self, scene, obj):
        filtered = []
        for _obj in self.scene:
            # if the x-coordinate of _obj is bigger than the x-coordinate of slef.currentObj,
            # then _obj is located to the right of self.currentObj
            if _obj["position"][0] > obj["position"][0] and _obj["id"] != obj["id"]:
                filtered.append(_obj)
        return filtered

    def filterFront(self, scene, obj):
        filtered = []
        if len(scene) == 0:
            return filtered

        for _obj in self.scene:
            # if the y-coordinate of _obj is smaller than the y-coordinate of slef.currentObj,
            # then _obj is located in front of self.currentObj
            if _obj["position"][1] > obj["position"][1] and _obj["id"] != obj["id"]:
                filtered.append(_obj)
        return filtered

    def filterBehind(self, scene, obj):
        # assert type(scene) == list, "Excpected type list got {} instead".format(type(scene))
        filtered = []
        if len(scene) == 0:
            return filtered

        for _obj in scene:
            # if the y-coordinate of _obj is bigger than the y-coordinate of slef.currentObj,
            # then _obj is located behind self.currentObj
            if _obj["position"][1] < obj["position"][1] and _obj["id"] != obj["id"]:
                filtered.append(_obj)
        return filtered

    def filterPosition(self, scene, obj, pos):
        # assert type(scene) == list, "Excpected type list got {} instead".format(type(scene))
        assert pos in ["left", "right", "front", "behind"]
        if pos == "left":
            filtered = self.filterLeft(scene, obj)
        elif pos == "right":
            filtered = self.filterRight(scene, obj)
        elif pos == "front":
            filtered = self.filterFront(scene, obj)
        elif pos == "behind":
            filtered = self.filterBehind(scene, obj)

        return filtered

    ###########################################################################
    #                           Counting questions                            #
    ###########################################################################
    def countAll(self):
        self.currentGrp = deepcopy(self.scene)
        self.groups.append(deepcopy(self.scene))
        return len(self.scene)

    def countOther(self):
        others = self.getOther()
        if len(others) > 0:
            self.currentGrp = others
            self.groups.append(others)
        if len(others) == 1:
            obj = others[0]
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    break
            self.updateCurrentObj(obj)

            self.updateVisited(obj)
        return len(others)

    def countAllGroup(self):
        return len(self.currentGrp)

    def countAttribute(self, attribute, updateCurrentObj=True):
        filtered = self.filterAttribute(self.scene, attribute)
        if len(filtered) == 0:
            return 0
        # Update the visited objects list
        for _obj in filtered:
            self.updateVisited(_obj)
        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            self.updateIdentifier(obj, attribute)
            self.updateVisited(obj)
            if updateCurrentObj:
                self.updateCurrentObj(obj)
            else:
                if new:
                    self.objs.append(obj)

        self.groups.append(filtered)
        self.currentGrp = filtered
        return len(filtered)

    def countAttributeGroup(self, attribute, updateCurrentObj=True):
        filtered = self.filterAttribute(self.currentGrp, attribute)
        if len(filtered) == 0:
            return 0
        # Update the visited objects list
        for _obj in filtered:
            self.updateVisited(_obj)
        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            self.updateIdentifier(obj, attribute)
            self.updateVisited(obj)

            if updateCurrentObj:
                self.updateCurrentObj(obj)
            else:
                if new:
                    self.objs.append(obj)

        self.groups.append(filtered)
        self.currentGrp = filtered
        return len(filtered)

    def countObjRelImm(self, pos, updateCurrentObj=True):
        filtered = self.filterPosition(self.scene, self.currentObj, pos)
        if len(filtered) == 0:
            return 0
        # Update the visited objects list
        for _obj in filtered:
            self.updateVisited(_obj)

        self.currentGrp = filtered
        self.groups.append(filtered)

        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            if updateCurrentObj:
                self.updateCurrentObj(obj)
                self.uniqueObjFlag = True
            else:
                if new:
                    self.objs.append(obj)
        return len(filtered)

    def countObjRelImm2(self, pos):
        if self.uniqueObjFlag:
            # del self.objs[-1]
            self.updateCurrentObj(self.objs[-2])
            self.uniqueObjFlag = False
        return self.countObjRelImm(pos)

    def countObjRelEarly(self, pos, earlyObjAttribute, updateCurrentObj=True):
        for objEarly in reversed(self.objs):
            if objEarly["identifier"] is not None:
                identifiers = objEarly["identifier"].split("-")
                if earlyObjAttribute in identifiers:
                    break
            else:
                continue
        filtered = self.filterPosition(self.scene, objEarly, pos)
        if len(filtered) == 0:
            return 0
        # Update the visited objects list
        for _obj in filtered:
            self.updateVisited(_obj)

        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            if updateCurrentObj:
                self.updateCurrentObj(obj)
            else:
                if new:
                    self.objs.append(obj)
        else:
            self.updateCurrentObj(objEarly)

        self.currentGrp = filtered
        self.groups.append(filtered)
        return len(filtered)

    def countObjExcludeImm(self, attributeType, updateCurrentObj=True):
        filtered = self.excludeAttribute(
            self.scene, self.currentObj, attributeType)
        if len(filtered) == 0:
            return 0

        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            if updateCurrentObj:
                self.updateCurrentObj(obj)
            else:
                if new:
                    self.objs.append(obj)

        self.currentGrp = filtered
        self.groups.append(filtered)
        return len(filtered)

    def countObjExcludeEarly(self, attributeType, earlyObjAttribute, updateCurrentObj=True):
        for objEarly in reversed(self.objs):
            if objEarly["identifier"] is not None:
                identifiers = objEarly["identifier"].split("-")
                if earlyObjAttribute in identifiers:
                    break
            else:
                continue

        filtered = self.excludeAttribute(self.scene, objEarly, attributeType)
        if len(filtered) == 0:
            return 0

        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj = _obj
                    new = False
                    break
            if updateCurrentObj:
                self.updateCurrentObj(obj)
            else:
                if new:
                    self.objs.append(obj)
        else:
            self.updateCurrentObj(objEarly)
        self.currentGrp = filtered
        self.groups.append(filtered)
        return len(filtered)

    ###########################################################################
    #                           Existence questions                           #
    ###########################################################################

    def existOther(self):
        others = self.getOther()
        numOther = len(others)
        if numOther > 0:
            self.currentGrp = others
            self.groups.append(others)
            for _obj in others:
                self.updateVisited(_obj)
        return "yes" if numOther > 0 else "no"

    def existAttribute(self, attribute):
        filtered = self.filterAttribute(self.scene, attribute)
        numAttribute = len(filtered)
        if numAttribute == 0:
            return "no"

        # Update the visited objects list
        for _obj in filtered:
            self.updateVisited(_obj)
        if len(filtered) == 1:
            obj = filtered[0]
            new = True
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    self.updateIdentifier(_obj, attribute)
                    new = False
                    break
            if new:
                self.updateIdentifier(obj, attribute)
                self.objs.append(obj)
                # self.updateCurrentObj(obj)

        self.currentGrp = filtered
        self.groups.append(filtered)
        return "yes"

    def existAttributeGroup(self, attribute):
        numAttributeGrp = self.countAttributeGroup(
            attribute, updateCurrentObj=False)
        return "yes" if numAttributeGrp > 0 else "no"

    def existObjRelImm(self, pos):
        numObjs = self.countObjRelImm(pos, updateCurrentObj=False)
        return "yes" if numObjs > 0 else "no"

    def existObjRelEarly(self, pos, earlyObjAttribute):
        numObjs = self.countObjRelEarly(
            pos, earlyObjAttribute, updateCurrentObj=False)
        return "yes" if numObjs > 0 else "no"

    def existObjExcludeImm(self, attributeType):
        numObjs = self.countObjExcludeImm(
            attributeType, updateCurrentObj=False)
        return "yes" if numObjs > 0 else "no"

    def existObjExcludeEarly(self, attributeType, earlyObjAttribute):
        for objEarly in reversed(self.objs):
            if objEarly["identifier"] is not None:
                identifiers = objEarly["identifier"].split("-")
                if earlyObjAttribute in identifiers:
                    break
            else:
                continue

        filtered = self.excludeAttribute(self.scene, objEarly, attributeType)
        numObjs = len(filtered)
        if numObjs == 0:
            return "no"
        self.currentGrp = filtered
        self.groups.append(filtered)
        return "yes"

    ###########################################################################
    #                             Seek questions                              #
    ###########################################################################

    def seekAttrImm(self, attributeType):
        assert attributeType in self.currentObj, "Attributre <{}> is not valid"
        self.updateIdentifier(self.currentObj, self.currentObj[attributeType])
        return self.currentObj[attributeType]

    def seekAttributeEarly(self, attributeType, earlyObjAttribute):
        for objEarly in reversed(self.objs):
            if objEarly["identifier"] is not None:
                identifiers = objEarly["identifier"].split("-")
                if earlyObjAttribute in identifiers:
                    break
            else:
                continue
        self.updateIdentifier(objEarly, objEarly[attributeType])
        self.updateCurrentObj(objEarly)
        self.updateVisited(objEarly)
        return objEarly[attributeType]

    def seekAttributeRelImm(self, attributeType, pos):
        filtered = self.filterPosition(self.scene, self.currentObj, pos)
        if len(filtered) == 0:
            return "none"
        else:
            # Get the closest object to slef.obj
            if pos == "left":
                filtered.sort(key=lambda x: x["position"][0])
                obj = filtered[-1]
            elif pos == "right":
                filtered.sort(key=lambda x: x["position"][0])
                obj = filtered[0]
            elif pos == "front":
                filtered.sort(key=lambda x: x["position"][1])
                obj = filtered[0]
            elif pos == "behind":
                filtered.sort(key=lambda x: x["position"][1])
                obj = filtered[-1]

            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj["identifier"] = _obj["identifier"]
                    break
            self.updateIdentifier(obj, obj[attributeType])
            self.updateCurrentObj(obj)
            self.updateVisited(obj)
            return obj[attributeType]

    def seekAttributeRelEarly(self, attributeType, pos, earlyObjAttribute):
        for objEarly in reversed(self.objs):
            if objEarly["identifier"] is not None:
                identifiers = objEarly["identifier"].split("-")
                if earlyObjAttribute in identifiers:
                    break
            else:
                continue

        filtered = self.filterPosition(self.scene, objEarly, pos)
        if len(filtered) == 0:
            return "none"
        else:
            # Get the closest object to slef.obj
            if pos == "left":
                filtered.sort(key=lambda x: x["position"][0])
                obj = filtered[-1]
            elif pos == "right":
                filtered.sort(key=lambda x: x["position"][0])
                obj = filtered[0]
            elif pos == "front":
                filtered.sort(key=lambda x: x["position"][1])
                obj = filtered[0]
            elif pos == "behind":
                filtered.sort(key=lambda x: x["position"][1])
                obj = filtered[-1]
            for _obj in self.objs:
                if _obj["id"] == obj["id"]:
                    obj["identifier"] = _obj["identifier"]
                    break
            self.updateIdentifier(obj, obj[attributeType])
            self.updateCurrentObj(obj)
            self.updateVisited(obj)
            return obj[attributeType]



In [41]:
import argparse
import os
import torch
#import utils_m

In [42]:
#!touch "pkl-small/quastion-train" res.txt

In [43]:
class OptionsC():#changed optiopn class as Option_c to differentiate it with the one belong to question
    def __init__(self):
        self.parser = argparse.ArgumentParser()
        self.initialized = False

    def initialize(self):
        self.parser.add_argument(
            '--mode',
            default="train",#*** in the training mode.
            # required=True,
            type=str,
            choices=['train', 'test'],
            help='The mode of the experiment')

        self.parser.add_argument(
            '--run_dir',
            
            default="pkl-small/quastion-train",#for saving the results of the small dataset
            # required=True,
            type=str,
            help='The experiment directory')

        self.parser.add_argument(
            '--load_checkpoint_path',
            
            default='None',#*** training mode
            type=str,
            help='The path the the pretrained CaptionNet')

        self.parser.add_argument(
            '--res_path',
            
            default="pkl-small/quastion-train",#for saving the results of the small dataset
            # required=True,
            type=str,
            help='Path where to log the predicted caption programs')

        self.parser.add_argument(
            '--gpu_ids',
            default='0',
            type=str,
            help='Id of the gpu to be used')

        self.parser.add_argument(
            '--seed',
            default=42,
            type=int,
            help='The seed used in training')

        self.parser.add_argument(
            '--dataPathTr',
            
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/train_concat_half.h5",
            # required=True,
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed training data')

        self.parser.add_argument(
            '--dataPathVal',
            
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/val_concat_half.h5",
            # required=True,
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed validation data')

        self.parser.add_argument(
            '--dataPathTest',
            # required=True,
            
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/test_concat_75000.h5",
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed test data')

        self.parser.add_argument(
            '--vocabPath',
            
            default = "/kaggle/input/nsvd-dataset/train_concat/vocab_output.json", 
            # required=True,
            type=str,
            help='Path to the generated vocabulary')

        self.parser.add_argument(
            '--batch_size',
            default=64,
            type=int,
            help='Batch size')

        self.parser.add_argument(
            '--num_workers',
            default=0,
            type=int,
            help='Number of workers for loading')

        self.parser.add_argument(
            '--num_iters',
            #default=5000,
            default=TOTAL_ITER,
            type=int,
            help='Total number of iterations')

        self.parser.add_argument(
            '--display_every',
            default=5,
            type=int,
            help='Display training information every N iterations')

        self.parser.add_argument(
            '--debug_every',
            default=100,
            type=int,
            help='Display debug message every N iterations')

        self.parser.add_argument(
            '--validate_every',
            default=VALID_EVE,
            #default=200,
            type=int,
            help='Validate every N iterations')

        self.parser.add_argument(
            '--shuffle_data',
            default=1,
            type=int,
            help='Activate to shuffle the training data')

        self.parser.add_argument(
            '--optim',
            default='adam',
            type=str,
            help='The name of the optimizer to be used')

        self.parser.add_argument(
            '--lr',
            default=1e-3, #A version
            #default=1e-1,#just change to be synced with the OptionQ
            type=float,
            help='Base learning rate')

        self.parser.add_argument(
            '--betas',
            default='0.9, 0.98',
            type=str,
            help='Adam optimizer\'s betas')

        self.parser.add_argument(
            '--eps',
            default='1e-9',
            type=float,
            help='Adam optimizer\'s epsilon')

        self.parser.add_argument(
            '--lr_decay_marks',
            default='50000, 55000',
            type=str,
            help='Learing rate decay marks')

        self.parser.add_argument(
            '--lr_decay_factor',
            #default=0.5,
            default=0.3,
            type=float,
            help='Learning rate decay factor')

        self.parser.add_argument(
            '--weight_decay',
            default=1e-6, # the original value
            #default=1e-2, # To reach to a better accuracy
            type=float,
            help='Weight decay')

        self.parser.add_argument(
            '--embedDim',
            default=300,
            type=int,
            help='Embedding dimension')

        self.parser.add_argument(
            '--hiddenDim',
            default=512,
            type=int,
            help='LSTM hidden dimension')

        self.parser.add_argument(
            '--numLayers',
            #default=2,
            default=1,#to sync with the ques train
            type=int,
            help='Number of hidden LSTM layers')

        self.parser.add_argument(
            '--dropout',
            #default=0.1, #original value
            default=0.2, # to make it sync with ques_train
            type=float,
            help='Dropout value')

        self.parser.add_argument(
            '--multiHead',
            default=8,
            type=int,
            help='Number of attention heads')

        self.parser.add_argument(
            '--hiddenSizeHead',
            default=64,
            type=int,
            help='Dimension of each attention head')

        self.parser.add_argument(
            '--FeedForwardSize',
            default=2048,
            type=int,
            help='Dimension of the feed forward layer')

        self.parser.add_argument(
            '--FlatMLPSize',
            default=512,
            type=int,
            help='MLP flatten size')

        self.parser.add_argument(
            '--FlatGlimpses',
            default=1,
            type=int,
            help='Number of flatten glimpses')

        self.parser.add_argument(
            '--FlatOutSize',
            default=512,
            type=int,
            help='Final attention reduction dimension')

        self.parser.add_argument(
            '--layers',
            default=6,#A'original
            #default=8,#to sync with questrain
            type=int,
            help='Number of self attention layers')

        self.parser.add_argument(
            '--bidirectional',
            default=1,
            type=int,
            help='Activate to use bidirectional LSTMs')

        self.initialized = True

    def parse(self):
        # initialize parser
        if not self.initialized:
            self.initialize()
       # self.opts = self.parser.parse_args()
        self.opts, unknown = self.parser.parse_known_args()#this is added by me to fix the error of command line arguments.

        # parse gpu id list
        str_gpu_ids = self.opts.gpu_ids.split(',')
        self.opts.gpu_ids = []
        for str_id in str_gpu_ids:
            if str_id.isdigit() and int(str_id) >= 0:
                self.opts.gpu_ids.append(int(str_id))
        if len(self.opts.gpu_ids) > 0 and torch.cuda.is_available():
            print('\n[INFO] Using {} CUDA device(s) ...'.format(len(self.opts.gpu_ids)))
        else:
            print('\n[INFO] Using cpu ...')
            self.opts.gpu_ids = []

        # parse the optimizer's betas and lr decay marks
        self.opts.betas = [float(beta) for beta in self.opts.betas.split(',')]
        lr_decay_marks = [int(m) for m in self.opts.lr_decay_marks.split(',')]
        for i in range(1, len(lr_decay_marks)):
            assert lr_decay_marks[i] > lr_decay_marks[i-1]
        self.opts.lr_decay_marks = lr_decay_marks

        # print and save options
        args = vars(self.opts)
        print('\n ' + 30*'-' + 'Opts' + 30*'-')
        # for k, v in args.items():
            # print('%s: %s' % (str(k), str(v)))

        if not os.path.isdir(self.opts.run_dir):
            os.makedirs(self.opts.run_dir)
        filename = 'opts_c.txt'
        file_path = os.path.join(self.opts.run_dir, filename)
        with open(file_path, 'wt') as fout:
            fout.write('| options\n')
            for k, v in sorted(args.items()):
                fout.write('%s: %s\n' % (str(k), str(v)))
        return self.opts


In [44]:
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [45]:
class FC(nn.Module):
    def __init__(self, in_size, out_size, dropout_r=0., use_relu=True):
        super(FC, self).__init__()
        self.dropout_r = dropout_r
        self.use_relu = use_relu

        self.linear = nn.Linear(in_size, out_size)

        if use_relu:
            self.relu = nn.ReLU(inplace=True)

        if dropout_r > 0:
            self.dropout = nn.Dropout(dropout_r)

    def forward(self, x):
        x = self.linear(x)

        if self.use_relu:
            x = self.relu(x)

        if self.dropout_r > 0:
            x = self.dropout(x)

        return x


In [46]:
class MLP(nn.Module):
    def __init__(self, in_size, mid_size, out_size, dropout_r=0., use_relu=True):
        super(MLP, self).__init__()

        self.fc = FC(in_size, mid_size, dropout_r=dropout_r, use_relu=use_relu)
        self.linear = nn.Linear(mid_size, out_size)

    def forward(self, x):
        return self.linear(self.fc(x))

In [47]:
class LayerNorm(nn.Module):
    def __init__(self, size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.eps = eps

        self.a_2 = nn.Parameter(torch.ones(size))
        self.b_2 = nn.Parameter(torch.zeros(size))

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)

        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [48]:
class MHAtt(nn.Module):
    def __init__(self, opts):
        super(MHAtt, self).__init__()
        self.opts = opts

        self.linear_v = nn.Linear(opts.hiddenDim, opts.hiddenDim)
        self.linear_k = nn.Linear(opts.hiddenDim, opts.hiddenDim)
        self.linear_q = nn.Linear(opts.hiddenDim, opts.hiddenDim)
        self.linear_merge = nn.Linear(opts.hiddenDim, opts.hiddenDim)

        self.dropout = nn.Dropout(opts.dropout)

    def forward(self, v, k, q, mask):
        n_batches = q.size(0)

        v = self.linear_v(v).view(
            n_batches,
            -1,
            self.opts.multiHead,
            self.opts.hiddenSizeHead
        ).transpose(1, 2)

        k = self.linear_k(k).view(
            n_batches,
            -1,
            self.opts.multiHead,
            self.opts.hiddenSizeHead
        ).transpose(1, 2)

        q = self.linear_q(q).view(
            n_batches,
            -1,
            self.opts.multiHead,
            self.opts.hiddenSizeHead
        ).transpose(1, 2)

        atted = self.att(v, k, q, mask)
        atted = atted.transpose(1, 2).contiguous().view(
            n_batches,
            -1,
            self.opts.hiddenDim
        )

        atted = self.linear_merge(atted)

        return atted

    def att(self, value, key, query, mask):
        d_k = query.size(-1)

        scores = torch.matmul(
            query, key.transpose(-2, -1)
        ) / math.sqrt(d_k)

        if mask is not None:
            scores = scores.masked_fill(mask, -1e9)

        att_map = F.softmax(scores, dim=-1)
        att_map = self.dropout(att_map)

        return torch.matmul(att_map, value)

In [49]:
class FFN(nn.Module):
    def __init__(self, opts):
        super(FFN, self).__init__()

        self.mlp = MLP(
            in_size=opts.hiddenDim,
            mid_size=opts.FeedForwardSize,
            out_size=opts.hiddenDim,
            dropout_r=opts.dropout,
            use_relu=True
        )

    def forward(self, x):
        return self.mlp(x)


In [50]:
class SA(nn.Module):
    def __init__(self, opts):
        super(SA, self).__init__()
        self.mhatt = MHAtt(opts)
        self.ffn = FFN(opts)

        self.dropout1 = nn.Dropout(opts.dropout)
        self.norm1 = LayerNorm(opts.hiddenDim)

        self.dropout2 = nn.Dropout(opts.dropout)
        self.norm2 = LayerNorm(opts.hiddenDim)

    def forward(self, x, x_mask):
        x = self.norm1(x + self.dropout1(
            self.mhatt(x, x, x, x_mask)
        ))

        x = self.norm2(x + self.dropout2(
            self.ffn(x)
        ))

        return x

In [51]:
class AttFlat(nn.Module):
    def __init__(self, opts):
        super(AttFlat, self).__init__()
        self.opts = opts

        self.mlp = MLP(
            in_size=opts.hiddenDim,
            mid_size=opts.FlatMLPSize,
            out_size=opts.FlatGlimpses,
            dropout_r=opts.dropout,
            use_relu=True
        )
        # FLAT_GLIMPSES = 1
        self.linear_merge = nn.Linear(
            opts.hiddenDim * opts.FlatGlimpses,
            opts.FlatOutSize
        )

    def forward(self, x, x_mask):
        att = self.mlp(x)
        att = att.masked_fill(
            x_mask.squeeze(1).squeeze(1).unsqueeze(2),
            -1e9
        )
        att = F.softmax(att, dim=1)

        att_list = []
        for i in range(self.opts.FlatGlimpses):
            att_list.append(
                torch.sum(att[:, :, i: i + 1] * x, dim=1)
            )

        x_atted = torch.cat(att_list, dim=1)
        x_atted = self.linear_merge(x_atted)

        return x_atted

In [52]:
class Encoder(nn.Module):
    def __init__(self, opts, textVocabSize):
        super(Encoder, self).__init__()

        self.embedding_Cap = nn.Embedding(textVocabSize, opts.embedDim)
        bidirectional = opts.bidirectional > 0
        self.lstmC = nn.LSTM(
            input_size=opts.embedDim,
            hidden_size=opts.hiddenDim,
            num_layers=opts.numLayers,
            batch_first=True,
            bidirectional=bidirectional
        )


        self.embedding_Quest = nn.Embedding(textVocabSize, opts.embedDim)
        self.lstmQ = nn.LSTM(
            input_size=opts.embedDim,
            hidden_size=opts.hiddenDim,
            num_layers=opts.numLayers,
            bidirectional=bidirectional,
            batch_first=True
        )

        self.lstmH = nn.LSTM(
            input_size=opts.embedDim,
            hidden_size=opts.hiddenDim,
            num_layers=opts.numLayers,
            bidirectional=bidirectional,
            batch_first=True)
#++++++++++++++++++++++++++++++++++++++++++++++++++++++
        if bidirectional:
            opts.hiddenDim *= 2
            opts.hiddenSizeHead *= 2
            opts.FlatOutSize *= 2

        self.attCap = nn.ModuleList([SA(opts) for _ in range(opts.layers)])
        self.attFlatCap = AttFlat(opts)
        self.fc_Cap = nn.Linear(opts.hiddenDim, opts.hiddenDim)
#++++++++++++++++++++++++++++++++++++++++++++++++++++++
        self.attQues = nn.ModuleList([SA(opts) for _ in range(opts.layers)])
        self.attHist = nn.ModuleList([SA(opts) for _ in range(opts.layers)])

        self.attFlatQuest = AttFlat(opts)
        self.fc_Quest = nn.Linear(2 * opts.hiddenDim, opts.hiddenDim)


    def capForward(self, cap, hist=None):
        capMask = self.make_mask(cap.unsqueeze(2))
        cap = self.embedding_Cap(cap)
        cap, (_, _) = self.lstmC(cap)
        capO = cap.detach().clone()

        for attC in self.attCap:
            cap = attC(cap, capMask)
        # (batchSize, 512)
        cap = self.attFlatCap(cap, capMask)
        encOut = self.fc_Cap(cap)
        return encOut, capO
    
    def questForward(self, quest, hist):
        questMask = self.make_mask(quest.unsqueeze(2))
        histMask = self.make_mask(hist.unsqueeze(2))

        # quest = F.tanh(self.embedding(quest))
        quest = self.embedding_Quest(quest)

        quest, (_, _) = self.lstmQ(quest)
        questO = quest.detach().clone()

        hist = self.embedding_Quest(hist)
        hist, (_, _) = self.lstmH(hist)

        for attQ, attH in zip(self.attQues, self.attHist):
            quest = attQ(quest, questMask)
            hist = attH(hist, histMask)
        # (batchSize, 512)
        quest = self.attFlatQuest(quest, questMask)

        # hist: (batchSize, length, 512)
        attWeights = torch.sum(torch.mul(hist, quest.unsqueeze(1)), -1)
        attWeights = torch.softmax(attWeights, -1)
        hist = torch.sum(torch.mul(hist, attWeights.unsqueeze(2)), 1)
        encOut = self.fc_Quest(torch.cat([quest, hist], -1))

        return encOut, questO

    def forward(self, cap, quest, hist):
        capEncOut, capO = self.capForward(cap)
        questEncOut, questO = self.questForward(quest, hist)

        return capEncOut, questEncOut, capO, questO

    # Masking
    def make_mask(self, feature):
        return (torch.sum(
            torch.abs(feature),
            dim=-1
        ) == 0).unsqueeze(1).unsqueeze(2)

In [53]:
#*** ->for qiansu: copy and paste the whole class
class Decoder(nn.Module):
    def __init__(self, opts, progVocabSize, maxLen, startID=1, endID=2):
        super(Decoder, self).__init__()
        self.numLayers = opts.numLayers
        self.bidirectional = opts.bidirectional > 0
        self.maxLen = maxLen
        self.startID = startID
        self.endID = endID

        self.embedding = nn.Embedding(progVocabSize, opts.embedDim)
        self.lstmProg = nn.LSTM(
            input_size=opts.embedDim,
            hidden_size=2*opts.hiddenDim if self.bidirectional else opts.hiddenDim,
            num_layers=opts.numLayers,
            batch_first=True,
            #bidirectional=self.bidirectional,#???????
        )
        hiddenDim = opts.hiddenDim
        if self.bidirectional:
            hiddenDim *= 2

        self.fcAtt = nn.Linear(2*hiddenDim, hiddenDim)
        self.fcOut = nn.Linear(hiddenDim, progVocabSize)

    def initPrgHidden(self, encOut):
        hidden = [encOut for _ in range(self.numLayers)]
        hidden = torch.stack(hidden, 0).contiguous()
        return hidden, hidden

    def forwardStep(self, prog, progH, questO):
        #**********************************************our error relates to this prog cause in our case it is not acting as tensor anymore.
        batchSize = prog.size(0)
        inputDim = questO.size(1)
        prog = self.embedding(prog)
        outProg, progH = self.lstmProg(prog, progH)

        att = torch.bmm(outProg, questO.transpose(1, 2))
        att = F.softmax(att.view(-1, inputDim), 1).view(batchSize, -1, inputDim)
        context = torch.bmm(att, questO)
        # (batchSize, progLength, hiddenDim)
        out = F.tanh(self.fcAtt(torch.cat([outProg, context], dim=-1)))

        # (batchSize, progLength, progVocabSize)
        out = self.fcOut(out)
        predSoftmax = F.log_softmax(out, 2)
        return predSoftmax, progH

    def forward(self, prog, encOut, questO):
        progH = self.initPrgHidden(encOut)
        predSoftmax, progH = self.forwardStep(prog, progH, questO)

        return predSoftmax, progH

    def sample(self, encOut, questO):
        batchSize = encOut.size(0)
        cudaFlag = encOut.is_cuda
        progH = self.initPrgHidden(encOut)
        # prog = progCopy[:, 0:3]
        prog = torch.LongTensor(batchSize, 1).fill_(self.startID)
        # prog = torch.cat((progStart, progEnd), -1)
        if cudaFlag:
            prog = prog.cuda()
        outputLogProbs = []
        outputTokens = []
     

        def decode(i, output):
            tokens = output.topk(1, dim=-1)[1].view(batchSize, -1)
            #print("This is inside of the decode local function and this is the tockens=", tokens)
            return tokens

        for i in range(self.maxLen):
            predSoftmax, progH = self.forwardStep(prog, progH, questO)
            prog = decode(i, predSoftmax)
            prog_flat = list(chain(*prog))
            flat_list = [item.item() for item in prog_flat]

        #****************************************my modification
            outputTokens.append(flat_list)#new
       #print("lets check what is inside outputTocken", outputTokens)    
       # print("-----------------------------------------")
        return outputTokens, outputLogProbs


In [54]:
class SeqToSeqUnified(nn.Module):
    def __init__(self, encoder, decoder):
        super(SeqToSeqUnified, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, cap, quest, questProg, capProg, hist):
        capEncOut, questEncOut, capO, questO = self.encoder(cap, quest, hist)

        # unitEncOut = torch.cat((capEncOut, questEncOut), dim = 0)
        #++++++++++++++++++++++++
        #print("The shape of the capProg", capProg.shape)
        #print("The shape of the questProg", questProg.shape)
        #--------------------------
        #capProg = capProg[:, :-1]
        # unitO = torch.cat((capO, questO), dim = 0)
        # unitProg = torch.cat((capProg, questProg), dim = 0)

        # predSoftmax, progHC = self.decoder(unitProg, unitEncOut, unitO)
        predSoftmaxCap, progHCCap = self.decoder(capProg, capEncOut, capO)
        predSoftmaxQuest, progHCQuest = self.decoder(questProg, questEncOut, questO)
        # [changed]
        predSoftmax = torch.cat((predSoftmaxCap, predSoftmaxQuest), dim = 1)
        
        # predSoftmax = torch.cat((predSoftmaxCap, predSoftmaxQuest), dim = 0)
        return predSoftmax, progHCCap

    def sample(self, cap, quest, hist):
        with torch.no_grad():
            capEncOut, questEncOut, capO, questO  = self.encoder(cap, quest, hist)

            outputTokensC, outputLogProbsC = self.decoder.sample(capEncOut, capO)
            outputTokensQ, outputLogProbsQ = self.decoder.sample(questEncOut, questO)
            
        outputTokens_t_c = [[row[i] for row in outputTokensC] for i in range(len(outputTokensC[0]))]
        outputTokens_t_q = [[row[i] for row in outputTokensQ] for i in range(len(outputTokensQ[0]))]

        outputTokens_t = torch.cat((torch.tensor(outputTokens_t_c), torch.tensor(outputTokens_t_q)), dim = 1)

        return outputTokens_t

In [55]:
import torch
import torch.optim as Optim
from itertools import chain #***

In [56]:
class WarmupOptimizer(object):
    def __init__(self, lr_base, optimizer, data_size, batch_size):
        self.optimizer = optimizer
        self._step = 0
        self.lr_base = lr_base
        self._rate = 0
        self.data_size = data_size
        self.batch_size = batch_size

    def step(self):
        self._step += 1

        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate

        self.optimizer.step()

    def zero_grad(self):
        self.optimizer.zero_grad()

    def rate(self, step=None):
        if step is None:
            step = self._step

        if step <= int(self.data_size / self.batch_size * 1):
            r = self.lr_base * 1/2.
        else:
            r = self.lr_base

        return r


def get_optim(opts, model, data_size, lr_base=None):
    if lr_base is None:
        lr_base = opts.lr

    if opts.optim == 'adam':
        optim = Optim.Adam(
                filter(lambda p: p.requires_grad, model.parameters()),
                lr=0,
                betas=opts.betas,
                eps=opts.eps,

            )
    elif opts.optim == 'rmsprop':
        optim = Optim.RMSprop(
                filter(lambda p: p.requires_grad, model.parameters()),
                lr=0,
                eps=opts.eps,
                weight_decay=opts.weight_decay
            )
    else:
        raise ValueError('{} optimizer is not supported'.fromat(opts.optim))
    return WarmupOptimizer(
        lr_base,
        optim,
        data_size,
        opts.batch_size
    )

def adjust_lr(optim, decay_r):
    optim.lr_base *= decay_r


In [57]:
import argparse
import os
#import utils_m
import torch

In [58]:
class OptionsQ():#changed optiopn class as Option_q to differentiate it with the one belong to caption
    def __init__(self):
        self.parser = argparse.ArgumentParser()
        self.initialized = False
        

    def initialize(self):
        self.parser.add_argument(
            '--mode',
            #required=True,
            default='test_with_gt',#***-> this is for train-concat and train-stack
            type=str,
            help='The mode of the experiment')
        self.parser.add_argument(
            '--type',
            default= 'q',
            help='The encoder type is caption encoder.'
        )

        self.parser.add_argument(
            '--run_dir',
            default = "/kaggle/working/", 
            type=str,
            help='The experiment directory')
        #***
        self.parser.add_argument(
            '--useCuda',
            default=1,
            type=int,
            help='hahahaha')
         #***
        self.parser.add_argument(
            '--text_log_dir',
            default="pkl-small/quastion-train",#for saving the results of the small dataset
            type=str,
            help='File to save the logged text')

        self.parser.add_argument(
            '--questionNetPath',
            default="None",#fixed this it should have not been None.
            type=str,
            help='Path to the pretrained QuestionNet that will be used for testing.')
        #*********
        self.parser.add_argument(
            '--trainedUnifiedNetPath',
            default = "/kaggle/input/nsvd-dataset/ckpt_iter_50.0_100.pkl",
            type=str,
            help='Path to the pretrained trainedUnifiedNetPath that will be used for testing.')
        #*********

        self.parser.add_argument(
            '--captionNetPath',
            default= 'None',
            type=str,
            help='Path to the pretrained CaptionNet that will be used for testing.')

        self.parser.add_argument(
            '--dialogLen',
            default=10,
            type=int,
            help='Length of the dialogs to be used for testing. We used 10, 15, and 20 in our experiments.')

        self.parser.add_argument(
            '--last_n_rounds',
            default=10,
            type=int,
            help='Number of the last rounds to consider in the history. We used 1, 2, 3, 4, and 10 in our experiments. ')

        self.parser.add_argument(
            '--encoderType',
            #required=True,
            default=1,#***-> this is for question-concat
            #default= 2, #*** -> this one is for question-stack
            type=int,
            choices=[1, 2],
            help='Type of the encoder: 1 --> Concat, 2 --> Stack')

        self.parser.add_argument(
            '--load_checkpoint_path',
            default='None',
            type=str,
            help='Path to a QestionNet checkpoint path to resume training')

        self.parser.add_argument(
            '--gpu_ids',
            default='0',
            type=str,
            help='Id of the gpu to be used')

        self.parser.add_argument(
            '--seed',
            default=42,
            type=int,
            help='The seed used in training')

        self.parser.add_argument(
            '--dataPathTr',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/train_concat_half.h5", 
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed training data')

        self.parser.add_argument(
            '--dataPathVal',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/val_concat_half.h5", 
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed validation data')

        self.parser.add_argument(
            '--dataPathTest',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/Small_Tr_Val_Test_Final/test_concat_75000.h5",
            type=str,
            help='Path to the h5 file of the Clevr-Dialog preprocessed test data')

        self.parser.add_argument(
            '--scenesPath',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/data/CLEVR_scenes_test.json", 
            type=str,
            help='Path to the derendered clevr-dialog scenes')

        self.parser.add_argument(
            '--vocabPath',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/train_concat/vocab_output.json", 
            type=str,
            help='Path to the generated vocabulary')
        
        self.parser.add_argument(
            '--vocabTestPath',
            #required=True,
            default = "/kaggle/input/nsvd-dataset/test_concat/vocab_output.json",
            type=str,
            help='Path to the generated vocabulary')

        self.parser.add_argument(
            '--batch_size',
            default=64,
            type=int,
            help='Batch size')

        self.parser.add_argument(
            '--countFirstFailueRound',
            default=0,
            type=int,
            help='If activated, we count the first failure round')

        self.parser.add_argument(
            '--maxSamples',
            default=-1,
            type=int,
            help='Maximum number of training samples')

        self.parser.add_argument(
            '--num_workers',
            default=0,
            type=int,
            help='Number of workers for loading')

        self.parser.add_argument(
            '--num_iters',
            default=TOTAL_ITER,
            type=int,
            help='Total number of iterations')

        self.parser.add_argument(
            '--display_every',
            default=5,
            type=int,
            help='Display training information every N iterations')

        self.parser.add_argument(
            '--validate_every',
            default=VALID_EVE,
            #default=200,
            type=int,
            help='Validate every N iterations')

        self.parser.add_argument(
            '--shuffle_data',
            default=1,
            type=int,
            help='Activate to shuffle the training data')

        self.parser.add_argument(
            '--optim',
            default='adam',
            type=str,
            help='The name of the optimizer to be used')

        self.parser.add_argument(
            '--lr',
            default=1e-3,
            #default=1e-1, # to reach 51
            type=float,
            help='Base learning rate')

        self.parser.add_argument(
            '--betas',
            default='0.9, 0.98',
            type=str,
            help='Adam optimizer\'s betas')

        self.parser.add_argument(
            '--eps',
            default='1e-9',
            type=float,
            help='Adam optimizer\'s epsilon')

        self.parser.add_argument(
            '--lr_decay_marks',
            default='50000, 55000',
            type=str,
            help='Learing rate decay marks')

        self.parser.add_argument(
            '--lr_decay_factor',
            #default=0.5,
            default=0.3, #just trying
            type=float,
            help='Learning rate decay factor')

        self.parser.add_argument(
            '--weight_decay',
            default=1e-6,#original
            #default=1e-2,#for reaching more than 51.72 pr:1e-5
            type=float,
            help='Weight decay')

        self.parser.add_argument(
            '--embedDim',
            default=300,
            type=int,
            help='Embedding dimension')

        self.parser.add_argument(
            '--hiddenDim',
            default=512,
            type=int,
            help='LSTM hidden dimension')

        self.parser.add_argument(
            '--numLayers',
            default=2,
            # default=1,
            type=int,
            help='Number of hidden LSTM layers')

        self.parser.add_argument(
            '--dropout',
            default=0.1,
            # default=0.2,
            type=float,
            help='Dropout value')

        self.parser.add_argument(
            '--multiHead',
            default=8,
            type=int,
            help='Number of attention heads')

        self.parser.add_argument(
            '--hiddenSizeHead',
            default=64,
            type=int,
            help='Dimension of each attention head')

        self.parser.add_argument(
            '--FeedForwardSize',
            default=2048,
            type=int,
            help='Dimension of the feed forward layer')

        self.parser.add_argument(
            '--FlatMLPSize',
            default=512,
            type=int,
            help='MLP flatten size')

        self.parser.add_argument(
            '--FlatGlimpses',
            default=1,
            type=int,
            help='Number of flatten glimpses')

        self.parser.add_argument(
            '--FlatOutSize',
            default=512,
            type=int,
            help='Final attention reduction dimension')

        self.parser.add_argument(
            '--layers',
            default=6,
            #default=8, #to reach more than 51
            type=int,
            help='Number of self attention layers')

        self.parser.add_argument(
            '--bidirectional',
            default=1,
            type=int,
            help='Activate to use bidirectional LSTMs')

        self.initialized = True

    def parse(self):
        # initialize parser
        if not self.initialized:
            self.initialize()
        #self.opts = self.parser.parse_args()#***
        self.opts, unknown = self.parser.parse_known_args()#this is added by me to fix the error of command line arguments.
        # parse gpu id list
        str_gpu_ids = self.opts.gpu_ids.split(',')
        self.opts.gpu_ids = []
        for str_id in str_gpu_ids:
            if str_id.isdigit() and int(str_id) >= 0:
                self.opts.gpu_ids.append(int(str_id))
        if len(self.opts.gpu_ids) > 0 and torch.cuda.is_available():
            print('\n[INFO] Using {} CUDA device(s) ...'.format(
                len(self.opts.gpu_ids)))
        else:
            print('\n[INFO] Using cpu ...')
            self.opts.gpu_ids = []

        # parse the optimizer's betas and lr decay marks
        self.opts.betas = [float(beta) for beta in self.opts.betas.split(',')]
        lr_decay_marks = [int(m) for m in self.opts.lr_decay_marks.split(',')]
        for i in range(1, len(lr_decay_marks)):
            assert lr_decay_marks[i] > lr_decay_marks[i-1]
        self.opts.lr_decay_marks = lr_decay_marks

        # print and save options
        args = vars(self.opts)
        print('\n ' + 30*'-' + 'Opts' + 30*'-')
        # for k, v in args.items():
            # print('%s: %s' % (str(k), str(v)))

        if not os.path.isdir(self.opts.run_dir):
            os.makedirs(self.opts.run_dir)
        filename = 'opts.txt'
        file_path = os.path.join(self.opts.run_dir, filename)
        with open(file_path, 'wt') as fout:
            fout.write('| options\n')
            for k, v in sorted(args.items()):
                fout.write('%s: %s\n' % (str(k), str(v)))
        return self.opts


In [59]:
import os
import sys
import json, torch, pickle, copy, time
import numpy as np
import argparse
import torch.nn as nn
import torch.utils.data as Data
from tensorboardX import SummaryWriter
from copy import deepcopy
#from clevrDialog_dataset import ClevrDialogQuestionDataset
import pickle
from tqdm import tqdm

#sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

#from executor.symbolic_executor import SymbolicExecutorClevr, SymbolicExecutorMinecraft
#from models import SeqToSeqQ, QuestEncoder_1, QuestEncoder_2, Decoder, CaptionEncoder, SeqToSeqC
#from optim import get_optim, adjust_lr
#from options_caption_parser import Options as OptionsC
#from options_question_parser import Options as OptionsQ

In [60]:
class Execution:
    def __init__(self, optsQ):
        self.opts_Q = deepcopy(optsQ)
        if self.opts_Q.useCuda > 0 and torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            print("[INFO] Using GPU {} ...".format(torch.cuda.get_device_name(0)))
        else:
            print("[INFO] Using CPU ...")
            self.device = torch.device("cpu")

        self.loss_fn = torch.nn.NLLLoss().to(self.device)

        print("[INFO] Loading dataset ...")

        self.datasetTr = ClevrDialogUnifiedDataset(
            self.opts_Q.dataPathTr, self.opts_Q.vocabPath, "train", "All tr data")
        self.datasetVal = ClevrDialogUnifiedDataset(
            self.opts_Q.dataPathVal, self.opts_Q.vocabPath, "val", "All val data", train=False)
        self.datasetTest = ClevrDialogUnifiedDataset(
            self.opts_Q.dataPathTest, self.opts_Q.vocabTestPath, "test", "All val data", train=False)
           

        self.unitNet = constructNet(
            self.opts_Q,
            self.datasetTr.lenVocabText,
            self.datasetTr.lenVocabProg,
            self.datasetTr.maxLenProg,
        )

        # TODO: add argument "trainedUnifiedNetPath"
        '''
        if os.path.isfile(self.opts_Q.trainedUnifiedNetPath):
            print('Loading CaptionNet from {}'.format(self.opts_Q.trainedUnifiedNetPath))
            state_dict = torch.load(self.opts_Q.trainedUnifiedNetPath)['state_dict']
            self.unitNet.load_state_dict(state_dict)'''

        self.unitNet.to(self.device)

        # if os.path.isfile(self.opts_Q.load_checkpoint_path):
        #     print('Loading questNet from {}'.format(optsQ.load_checkpoint_path))
        #     state_dict = torch.load(self.opts_Q.load_checkpoint_path)['state_dict']
        #     self.questNet.load_state_dict(state_dict)
        total_params_curr = sum(p.numel() for p in self.unitNet.parameters() if p.requires_grad)
        print("The quest encoder has {} trainable parameters".format(total_params_curr))

        if "minecraft" in self.opts_Q.scenesPath:
            self.symbolicExecutor = SymbolicExecutorMinecraft(self.opts_Q.scenesPath)
        else:
            self.symbolicExecutor = SymbolicExecutorClevr(self.opts_Q.scenesPath)

        tb_path = os.path.join(self.opts_Q.run_dir, "tb_logdir")
        if not os.path.isdir(tb_path):
            os.makedirs(tb_path)

        self.ckpt_path = os.path.join(self.opts_Q.run_dir, "ckpt_dir")
        if not os.path.isdir(self.ckpt_path):
            os.makedirs(self.ckpt_path)
        if not os.path.isdir(self.opts_Q.text_log_dir):
            os.makedirs(self.opts_Q.text_log_dir)

        self.writer = SummaryWriter(tb_path)
        self.iter_val = 0

        #if os.path.isfile(self.opts_Q.dependenciesPath):
           # with open(self.opts_Q.dependenciesPath, "rb") as f:
                #self.dependencies = pickle.load(f)

 

    # Evaluation
    def eval_with_gt(self):
        # Define the multi-gpu training if needed
        all_pred_answers = []
        all_gt_answers = []
        all_quest_types = []
        all_penalties = []
        all_pred_programs = []
        all_gt_programs = []

        first_failure_round = 0
        total_correct = 0
        total_acc_pen = 0
        total = 0
        total_curr_prog_correct = 0

        if len(self.opts_Q.gpu_ids) > 1:
            self.unitNet = nn.DataParallel(self.unitNet, device_ids=self.opts_Q.gpu_ids)
        self.unitNet = self.unitNet.eval()
        if self.opts_Q.batch_size != self.opts_Q.dialogLen:
            print("[INFO] Changed batch size from {} to {}".format(self.opts_Q.batch_size, self.opts_Q.dialogLen))
            self.opts_Q.batch_size = self.opts_Q.dialogLen
        dataloader = Data.DataLoader(
            self.datasetTest,
            batch_size=self.opts_Q.batch_size,
            shuffle=False,
            num_workers=self.opts_Q.num_workers,
            pin_memory=False
        )
        _iterCur = 0
        _totalCur = len(dataloader)
        print("This the len of dataloader:", _totalCur)

        for step, (quest, questPrg,questRounds, history, historiesProg, answer, questImgIdx) in enumerate(dataloader):
            # print("\rEvaluation: [step %4d/%4d]" % (
            #     step + 1,
            #     int(data_size / self.opts_Q.batch_size),
            # ), end='          ')
            # if step >= 5000:
            #     break
            batchSize = quest.size(0)
            quest = quest.to(self.device)
            # dependecy = self.dependencies[step*batchSize:(step+1)*batchSize]

            if history.dim() == 3:
                caption = history.detach()
                caption = caption[:, 0, :]
                caption = caption[:, :16].to(self.device)
            elif history.dim() == 2:
                caption = history.detach()
                caption = caption[:, :16].to(self.device)
            if self.opts_Q.last_n_rounds < 10:
                last_n_rounds_batch = []
                for i, r in enumerate(questRounds.tolist()):
                    startIdx = max(r - self.opts_Q.last_n_rounds, 0)
                    endIdx = max(r, self.opts_Q.last_n_rounds)
                    if history.dim() == 3:
                        assert endIdx - startIdx == self.opts_Q.last_n_rounds
                        histBatch = history[i, :, :]
                        last_n_rounds_batch.append(histBatch[startIdx:endIdx, :])
                    elif history.dim() == 2:
                        startIdx *= 20
                        endIdx *= 20
                        histBatch = history[i, :]
                        temp = histBatch[startIdx:endIdx]
                        if r > self.opts_Q.last_n_rounds:
                            last_n_rounds_batch.append(torch.cat([torch.tensor([1]), temp, torch.tensor([2])], 0))
                        else:
                            last_n_rounds_batch.append(torch.cat([temp, torch.tensor([2, 0])], 0))
                history = torch.stack(last_n_rounds_batch, dim=0)

            history = history.to(self.device)
            questPrg = questPrg.to(self.device)
            historiesProg = historiesProg.tolist()
            questRounds = questRounds.tolist()
            answer = answer.tolist()
            answers = list(map(lambda a: self.datasetTest.vocab["idx_text_to_token"][a], answer))
            questImgIdx = questImgIdx.tolist()
            # if "minecraft" in self.opts_Q.scenesPath:
            #     questImgIdx = [idx - 1 for idx in questImgIdx]

            
            unifiedProgsToksPred = torch.tensor(self.unitNet.sample(caption, quest, history))
            # chunks = torch.split(torch.tensor(unifiedProgsToksPred), 2, dim=0)
            # print("[debug] unifiedProg:".format(unifiedProgsToksPred))
            capProgsToksPred = unifiedProgsToksPred[:, 6:]
            currProgsToksPred = unifiedProgsToksPred[:, :6]


            currProgsPred = decodeProg(currProgsToksPred, self.datasetTest.vocab["idx_prog_to_token"])
            capProgsPred = decodeProg(capProgsToksPred, self.datasetTest.vocab["idx_prog_to_token"])
            targetProgs = decodeProg(questPrg, self.datasetTest.vocab["idx_prog_to_token"], target=True)


            questTypes = [targetProg[0] for targetProg in targetProgs]
            # progHistories = getProgHistories(historiesProg[0], dataset.vocab["idx_prog_to_token"])
            progHistories = [getProgHistories(progHistToks, self.datasetTest.vocab["idx_prog_to_token"]) for progHistToks in historiesProg]
            pred_answers = []
            all_pred_programs.append([capProgsPred[0]] + currProgsPred)
            all_gt_programs.append([progHistories[0]] + (targetProgs))

            for i in range(batchSize):
                # if capProgsPred[i][0] == "extreme-center":
                #     print("bla")
                # print("idx = {}".format(questImgIdx[i]))
                ans = self.getPrediction(
                    currProgsPred[i],
                    capProgsPred[i],
                    progHistories[i],
                    questImgIdx[i]
                )
                # if ans == "Error":
                #     print(capProgsPred[i])
                pred_answers.append(ans)
            # print(pred_answers)
            correct = [1 if pred == ans else 0 for (pred, ans) in zip(pred_answers, answers)]
            correct_prog = [1 if pred == ans else 0 for (pred, ans) in zip(currProgsPred, targetProgs)]
            idx_false = np.argwhere(np.array(correct) == 0).squeeze(-1)
            if idx_false.shape[-1] > 0:
                first_failure_round += idx_false[0] + 1
            else:
                first_failure_round += self.opts_Q.dialogLen + 1

            correct = sum(correct)
            correct_prog = sum(correct_prog)
            total_correct += correct
            total_curr_prog_correct += correct_prog
            total += len(answers)
            all_pred_answers.append(pred_answers)
            all_gt_answers.append(answers)
            all_quest_types.append(questTypes)
            penalty = np.ones_like(answers)

            # penalty = np.zeros_like(answers)
            all_penalties.append(penalty)
            _iterCur += 1
            if _iterCur % self.opts_Q.display_every == 0:
                print("[Evaluation] step {0} / {1} | acc. = {2:.2f}".format(
                    _iterCur, _totalCur, 100.0 * (total_correct / total)))

        ffr = 1.0 * (first_failure_round/_totalCur)/(self.opts_Q.dialogLen + 1)

        textOut = "\n --------------- Average First Failure Round --------------- \n"
        textOut += "{} / {}".format(ffr, self.opts_Q.dialogLen)

        # print(total_correct, total)
        accuracy = total_correct / total
        vd_acc = total_acc_pen / total
        curr_prog_acc = total_curr_prog_correct / total
        textOut += "\n --------------- Overall acc. --------------- \n"
        textOut += "{}".format(100.0 * accuracy)
        textOut += "\n --------------- Overall VD acc. --------------- \n"
        textOut += "{}".format(100.0 * vd_acc)
        textOut += "\n --------------- quest Prog. Acc --------------- \n"
        textOut += "{}".format(100.0 * curr_prog_acc)
        textOut += get_per_round_acc(
            all_pred_answers, all_gt_answers, all_penalties)

        textOut += get_per_quest_type_acc(
            all_pred_answers, all_gt_answers, all_quest_types, all_penalties)

        # textOut += get_per_dependency_type_acc(
        #     all_pred_answers, all_gt_answers, all_penalties)

        textOut += "\n --------------- Done --------------- \n"
        print(textOut)


# Evaluation
 

    def getPrediction(self, currProgPred, capProgPred, historyProg, imgIndex):
        self.symbolicExecutor.reset(imgIndex)
        # if round one, execute the predicted caption program first then answer the quest
        if len(historyProg) == 1:
            captionFuncLabel = capProgPred[0]
            captionFuncArgs = capProgPred[1:]

            questFuncLabel = currProgPred[0]
            questFuncArgs = currProgPred[1:]

            try:
                _ = self.symbolicExecutor.execute(captionFuncLabel, captionFuncArgs)
            except:
                return "Error"

            try:
                predAnswer = self.symbolicExecutor.execute(questFuncLabel, questFuncArgs)
            except:
                return "Error"

        # If it is not the first round, we have to execute the program history and
        # then answer the quest.
        else:
            questFuncLabel = currProgPred[0]
            questFuncArgs = currProgPred[1:]
            for prg in historyProg:
                # prg = prg.split(" ")
                FuncLabel = prg[0]
                FuncArgs = prg[1:]
                try:
                    _ = self.symbolicExecutor.execute(FuncLabel, FuncArgs)
                except:
                    return "Error"

            try:
                predAnswer = self.symbolicExecutor.execute(questFuncLabel, questFuncArgs)
            except:
                return "Error"
        return str(predAnswer)

    def run(self, run_mode, epoch=None):
        self.set_seed(self.opts_Q.seed)
        if run_mode == 'test_with_gt':
            print('Testing with gt answers in history')
            print('Loading ckpt {}'.format(self.opts_Q.trainedUnifiedNetPath))
            state_dict = torch.load(self.opts_Q.trainedUnifiedNetPath)['state_dict']
            self.unitNet.load_state_dict(state_dict)
            self.eval_with_gt()
        else:
            exit(-1)

    def set_seed(self, seed):
        """Sets the seed for reproducibility.
        Args:
            seed (int): The seed used
        """
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        print('[INFO] Seed set to {}...'.format(seed))


def constructquestNet(opts_Q, lenVocabText, lenVocabProg, maxLenProg):
    decoder = Decoder(opts_Q, lenVocabProg, maxLenProg)
    if opts_Q.encoderType == 1:
        encoder = currEncoder_1(opts_Q, lenVocabText)
    elif opts_Q.encoderType == 2:
        encoder = currEncoder_2(opts_Q, lenVocabText)

    net = SeqToSeqQ(encoder, decoder)
    return net


def constructCaptionNet(opts_Q, lenVocabText, lenVocabProg, maxLenProg):
    decoder = Decoder(opts_Q, lenVocabProg, maxLenProg)
    encoder = CaptionEncoder(opts_Q, lenVocabText)
    net = SeqToSeqC(encoder, decoder)
    return net

def constructNet(opts_Q, lenVocabText, lenVocabProg, maxLenProg):
    decoder = Decoder(opts_Q, lenVocabProg, maxLenProg)
    encoder = Encoder(opts_Q, lenVocabText)
    net = SeqToSeqUnified(encoder, decoder)
    return net

def getProgHistories(progHistToks, prgIdxToToken):
    progHist = []
    temp = []
    for tok in progHistToks:
        if tok not in [0, 1, 2]:
            temp.append(prgIdxToToken[tok])
            # del progHistToks[i]
        if tok == 2:
            # del progHistToks[i]
            # progHist.append(" ".join(temp))
            progHist.append(temp)
            temp = []
    return progHist


def getHistoriesFromStack(histToks, textIdxToToken):
    histories = "\n"
    temp = []
    for i, roundToks in enumerate(histToks):
        for tok in roundToks:
            if tok not in [0, 1, 2]:
                temp.append(textIdxToToken[tok])
                # del progHistToks[i]
            if tok == 2:
                # del progHistToks[i]
                if i == 0:
                    histories += " ".join(temp) + ".\n"
                else:
                    histories += " ".join(temp[:-1]) + "? | {}.\n".format(temp[-1])
                # histories.append(temp)
                temp = []
                break
    return histories


def getHistoriesFromConcat(histToks, textIdxToToken):
    histories = []
    temp = []
    for tok in histToks:
        if tok not in [0, 1, 2]:
            temp.append(textIdxToToken[tok])
            # del progHistToks[i]
        if tok == 2:
            # del progHistToks[i]
            histories.append(" ".join(temp[:-1]) + "? | {}".format(temp[-1]))
            # histories.append(temp)
            temp = []
    return histories


def decodeProg(tokens, prgIdxToToken, target=False):
    
    if (target == True):#***
        tokensBatch = tokens.tolist()
    else:#***
        tokensBatch = tokens
    progsBatch = []
    for tokens in tokensBatch:
        prog = []
        for tok in tokens:
            if tok == 2:  # <END> has index 2
                break
            prog.append(prgIdxToToken.get(int(tok)))
            
        if target:
            prog = prog[1:]
        
        progsBatch.append(prog)

    return progsBatch


def printPred(predSoftmax, gts, prgIdxToToken):
    assert predSoftmax.size(0) == gts.size(0)
    tokens = predSoftmax.topk(1)[1].squeeze(-1)
    tokens = tokens.tolist()
    gts = gts.tolist()
    message = "\n ------------------------ \n"
    for token, gt in zip(tokens, gts):
        message += "Prediction: "
        for tok in token:
            message += prgIdxToToken.get(tok) + " "
        message += "\n Target   : "
        for tok in gt:
            message += prgIdxToToken.get(tok) + " "
        message += "\n ------------------------ \n"
    return message


def get_per_round_acc(preds, gts, penalties):
    res = {}
    for img_preds, img_gt, img_pen in zip(preds, gts, penalties):
        img_preds = list(img_preds)
        img_gt = list(img_gt)
        img_pen = list(img_pen)
        for i, (pred, gt, pen) in enumerate(zip(img_preds, img_gt, img_pen)):
            _round = str(i + 1)
            if _round not in res:
                res[_round] = {
                    "correct": 0,
                    "all": 0
                }
            res[_round]["all"] += 1
            if pred == gt:
                res[_round]["correct"] += 0.5** float(pen)

    textOut = "\n --------------- Per round Acc --------------- \n"
    for k in res:
        textOut += "{}: {} %\n".format(k, 100.0 * (res[k]["correct"]/res[k]["all"]))
    return textOut


def get_per_quest_type_acc(preds, gts, qtypes, penalties):
    res1 = {}
    res2 = {}

    for img_preds, img_gt, img_qtypes, img_pen in zip(preds, gts, qtypes, penalties):
        # img_preds = list(img_preds)
        # img_gt = list(img_gt)
        img_pen = list(img_pen)
        for pred, gt, temp, pen in zip(img_preds, img_gt, img_qtypes, img_pen):
            if temp not in res1:
                res1[temp] = {
                    "correct": 0,
                    "all": 0
                }
            temp_cat = temp.split("-")[0]
            if temp_cat not in res2:
                res2[temp_cat] = {
                    "correct": 0,
                    "all": 0
                }
            res1[temp]["all"] += 1
            res2[temp_cat]["all"] += 1

            if pred == gt:
                res1[temp]["correct"] += 0.5** float(pen)
                res2[temp_cat]["correct"] += 0.5** float(pen)

    textOut = "\n --------------- Per quest Type Acc --------------- \n"
    for k in res1:
        textOut += "{}: {} %\n".format(k, 100.0 * (res1[k]["correct"]/res1[k]["all"]))

    textOut += "\n --------------- Per quest Category Acc --------------- \n"
    for k in res2:
        textOut += "{}: {} %\n".format(k, 100.0 * (res2[k]["correct"]/res2[k]["all"]))
    return textOut


def decode(tokens, prgIdxToToken, target=False):
    if type(tokens) != list:
        tokens = tokens.tolist()

    progsBatch = []
    for token in tokens:
        questProg = []
        for tok in token:
            if tok == 2:  # <END> has index 2
                break
            questProg.append(prgIdxToToken.get(tok))
        if target:
            questProg = questProg[1:]
        # progsBatch.append(" ".join(questProg))
        progsBatch.append(questProg)
    return progsBatch

In [61]:
optsQ = OptionsQ().parse()#***


[INFO] Using 1 CUDA device(s) ...

 ------------------------------Opts------------------------------


In [62]:

exe = Execution(optsQ)#***


[INFO] Using GPU Tesla T4 ...
[INFO] Loading dataset ...
The quest encoder has 202484871 trainable parameters


In [63]:
exe.run('test_with_gt')
print("[INFO] Done ...")#***

[INFO] Seed set to 42...
Testing with gt answers in history
Loading ckpt /kaggle/input/nsvd-dataset/ckpt_iter_50.0_100.pkl
[INFO] Changed batch size from 64 to 10
This the len of dataloader: 7500


/tmp/ipykernel_34/2936983435.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  unifiedProgsToksPred = torch.tensor(self.unitNet.sample(caption, quest, history))


[Evaluation] step 5 / 7500 | acc. = 10.00
[Evaluation] step 10 / 7500 | acc. = 13.00
[Evaluation] step 15 / 7500 | acc. = 12.67
[Evaluation] step 20 / 7500 | acc. = 12.50
[Evaluation] step 25 / 7500 | acc. = 11.20
[Evaluation] step 30 / 7500 | acc. = 12.00
[Evaluation] step 35 / 7500 | acc. = 11.43
[Evaluation] step 40 / 7500 | acc. = 11.00
[Evaluation] step 45 / 7500 | acc. = 10.89
[Evaluation] step 50 / 7500 | acc. = 10.40
[Evaluation] step 55 / 7500 | acc. = 10.36
[Evaluation] step 60 / 7500 | acc. = 10.33
[Evaluation] step 65 / 7500 | acc. = 10.31
[Evaluation] step 70 / 7500 | acc. = 9.71
[Evaluation] step 75 / 7500 | acc. = 9.60
[Evaluation] step 80 / 7500 | acc. = 9.62
[Evaluation] step 85 / 7500 | acc. = 9.88
[Evaluation] step 90 / 7500 | acc. = 10.00
[Evaluation] step 95 / 7500 | acc. = 10.00
[Evaluation] step 100 / 7500 | acc. = 9.90
[Evaluation] step 105 / 7500 | acc. = 9.90
[Evaluation] step 110 / 7500 | acc. = 9.82
[Evaluation] step 115 / 7500 | acc. = 9.83
[Evaluation] ste